# <img style="float: left; padding-right: 0.7cm; width: 3.0cm" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/179px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png">IIC2440 - Procesamiento de Datos Masivos
Pontificia Universidad Católica de Chile<br>
Semestre 2023-1<br>
Profesores: Adrián Soto y Juan Reutter <br>
Estudiantes: Eduardo Alvarez F y Javier Arriagada S

----

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=989165268aeb2b7bb3d3395e5e7eeaaba54973d86c6343d88c78dec4574ae928
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [7]:
from pyspark import SparkConf, SparkContext
import numpy as np
import networkx as nx
import random

# 1. Page Rank

In [3]:
conf = SparkConf().setMaster("local").setAppName("PageRank")
sc = SparkContext(conf = conf)

Generamos un tamaño n de nodos, a partir de los cuales creamos n3/2 de aristas. (Para no importar un archivo, aunque igual se podria hacer de esa manera)

In [8]:
num_nodes = 100
nodes = list(range(1,num_nodes +1))
G = nx.Graph()
G.add_nodes_from(range(1, num_nodes + 1))

# el 3/2 es solo por facilidad. Aumentar de ser necesario
num_edges = int(num_nodes *3/2)
edges = []
for _ in range(num_edges):
    node1 = random.randint(1, num_nodes)
    node2 = random.randint(1, num_nodes)
    while node2 == node1 or G.has_edge(node1, node2):
      node2 = random.randint(1, num_nodes)
    G.add_edge(node1, node2)

    edges.append((node1, node2))

In [9]:
# Crear RDD para nodos y edges
nodes_rdd = sc.parallelize(nodes)
edges_rdd = sc.parallelize(edges)

n  = int(nodes_rdd.count())

# Inicializar PageRank para cada nodo
initial_pagerank = 1/n
node_pageranks_rdd = nodes_rdd.map(lambda node: (node, initial_pagerank))

# Mostrar el RDD con los PageRanks iniciales
node_pageranks_rdd.collect()

[(1, 0.01),
 (2, 0.01),
 (3, 0.01),
 (4, 0.01),
 (5, 0.01),
 (6, 0.01),
 (7, 0.01),
 (8, 0.01),
 (9, 0.01),
 (10, 0.01),
 (11, 0.01),
 (12, 0.01),
 (13, 0.01),
 (14, 0.01),
 (15, 0.01),
 (16, 0.01),
 (17, 0.01),
 (18, 0.01),
 (19, 0.01),
 (20, 0.01),
 (21, 0.01),
 (22, 0.01),
 (23, 0.01),
 (24, 0.01),
 (25, 0.01),
 (26, 0.01),
 (27, 0.01),
 (28, 0.01),
 (29, 0.01),
 (30, 0.01),
 (31, 0.01),
 (32, 0.01),
 (33, 0.01),
 (34, 0.01),
 (35, 0.01),
 (36, 0.01),
 (37, 0.01),
 (38, 0.01),
 (39, 0.01),
 (40, 0.01),
 (41, 0.01),
 (42, 0.01),
 (43, 0.01),
 (44, 0.01),
 (45, 0.01),
 (46, 0.01),
 (47, 0.01),
 (48, 0.01),
 (49, 0.01),
 (50, 0.01),
 (51, 0.01),
 (52, 0.01),
 (53, 0.01),
 (54, 0.01),
 (55, 0.01),
 (56, 0.01),
 (57, 0.01),
 (58, 0.01),
 (59, 0.01),
 (60, 0.01),
 (61, 0.01),
 (62, 0.01),
 (63, 0.01),
 (64, 0.01),
 (65, 0.01),
 (66, 0.01),
 (67, 0.01),
 (68, 0.01),
 (69, 0.01),
 (70, 0.01),
 (71, 0.01),
 (72, 0.01),
 (73, 0.01),
 (74, 0.01),
 (75, 0.01),
 (76, 0.01),
 (77, 0.01),
 (78, 0.

In [10]:
# Obtener los vecinos de cada nodo
node_neighbors_rdd = edges_rdd.groupByKey()

In [11]:
def prepare_message(node, neighbors, pagerank):
  n = len(neighbors)
  messages = list()
  if n > 0:
    for neighbor in neighbors:
      message = (neighbor, pagerank / n)
      messages.append(message)

  return messages

In [12]:
messages_rdd = node_pageranks_rdd.join(node_neighbors_rdd).flatMap(lambda node: prepare_message(node[0], node[1][1], node[1][0])).collect()

incoming = list(set([message[0] for message in messages_rdd]))
non_incoming = [node for node in nodes if node not in incoming]

for node in non_incoming:
  messages_rdd.append((node, 0))

messages_rdd = sc.parallelize(messages_rdd)

In [13]:
node_received_messages_rdd = messages_rdd.reduceByKey(lambda x, y: x + y)
node_received_messages_rdd.collect()

[(7, 0.035),
 (97, 0.015),
 (29, 0.01),
 (100, 0.014166666666666668),
 (50, 0.025),
 (26, 0.015),
 (72, 0.008333333333333333),
 (10, 0.019166666666666665),
 (3, 0.0033333333333333335),
 (43, 0.015),
 (36, 0.008333333333333333),
 (19, 0.013333333333333332),
 (44, 0.013333333333333332),
 (86, 0.011666666666666667),
 (47, 0.0225),
 (12, 0.01),
 (89, 0.011666666666666667),
 (95, 0.016666666666666666),
 (16, 0.015833333333333335),
 (53, 0.0075),
 (51, 0.0025),
 (74, 0.0025),
 (76, 0.018666666666666668),
 (28, 0.004),
 (68, 0.0045000000000000005),
 (41, 0.0145),
 (46, 0.002),
 (84, 0.0175),
 (9, 0.005833333333333334),
 (40, 0.02),
 (18, 0.0325),
 (59, 0.018333333333333333),
 (80, 0.0033333333333333335),
 (65, 0.018333333333333333),
 (66, 0.005833333333333334),
 (71, 0.008333333333333333),
 (32, 0.005),
 (92, 0.015),
 (15, 0.01),
 (79, 0.01),
 (30, 0.008333333333333333),
 (5, 0.01),
 (6, 0.0033333333333333335),
 (20, 0.011666666666666667),
 (54, 0.01),
 (62, 0.01),
 (58, 0.01),
 (34, 0.0025),

In [14]:
damping_factor = 0.85

def update_pagerank(node, pagerank, damping_factor, messages, n):
    updated_pagerank = (1 - damping_factor)/n + damping_factor * messages
    return (node, updated_pagerank)

node_updated_pageranks_rdd = node_received_messages_rdd.join(node_pageranks_rdd).map(lambda node: update_pagerank(node[0], node[1][1], damping_factor, node[1][0], n))

In [15]:
node_updated_pageranks_rdd.collect()

[(100, 0.013541666666666667),
 (50, 0.022750000000000003),
 (26, 0.014249999999999999),
 (72, 0.008583333333333333),
 (10, 0.017791666666666667),
 (36, 0.008583333333333333),
 (44, 0.012833333333333332),
 (86, 0.011416666666666667),
 (12, 0.01),
 (16, 0.014958333333333334),
 (74, 0.0036250000000000006),
 (76, 0.01736666666666667),
 (28, 0.0049),
 (68, 0.005325000000000001),
 (46, 0.0032),
 (84, 0.016375),
 (40, 0.018500000000000003),
 (18, 0.029125),
 (80, 0.004333333333333334),
 (66, 0.006458333333333334),
 (32, 0.005750000000000001),
 (92, 0.014249999999999999),
 (30, 0.008583333333333333),
 (6, 0.004333333333333334),
 (20, 0.011416666666666667),
 (54, 0.01),
 (62, 0.01),
 (58, 0.01),
 (34, 0.0036250000000000006),
 (8, 0.0036250000000000006),
 (2, 0.012125),
 (4, 0.005750000000000001),
 (38, 0.006033333333333334),
 (98, 0.0071666666666666675),
 (88, 0.012833333333333334),
 (14, 0.005750000000000001),
 (64, 0.0117),
 (42, 0.0032),
 (52, 0.004333333333333334),
 (96, 0.00575000000000000

In [16]:
max_iterations = 20
epsilon = 0.0001

for iteration in range(max_iterations):

  messages_rdd = node_pageranks_rdd.join(node_neighbors_rdd).flatMap(lambda node: prepare_message(node[0], node[1][1], node[1][0])).collect()
  incoming = list(set([message[0] for message in messages_rdd]))
  non_incoming = [node for node in nodes if node not in incoming]
  for node in non_incoming:
    messages_rdd.append((node, 0))
  messages_rdd = sc.parallelize(messages_rdd)

  node_received_messages_rdd = messages_rdd.reduceByKey(lambda x, y: x + y)
  node_pageranks_rdd = node_received_messages_rdd.join(node_pageranks_rdd).map(lambda node: update_pagerank(node[0], node[1][1], damping_factor, node[1][0], n))

  if iteration > 0:
        prev_page_ranks_rdd = pageranks_rdd
        diff = prev_page_ranks_rdd.join(node_pageranks_rdd).mapValues(lambda ranks: abs(ranks[0] - ranks[1])).values().sum()

        if diff < epsilon:
          break

  pageranks_rdd = node_pageranks_rdd

In [17]:
final_page_ranks = pageranks_rdd.collect()
for node, rank in final_page_ranks:
    print("Node: {}, PageRank: {}".format(node, rank))

Node: 38, PageRank: 0.003636807113222504
Node: 95, PageRank: 0.010963235891321971
Node: 19, PageRank: 0.006658541757603916
Node: 76, PageRank: 0.005849762548377464
Node: 57, PageRank: 0.0015000000000000002
Node: 96, PageRank: 0.0021375000000000005
Node: 1, PageRank: 0.00554178383369446
Node: 20, PageRank: 0.0042865930749021106
Node: 58, PageRank: 0.006992010517768602
Node: 39, PageRank: 0.0015000000000000002
Node: 77, PageRank: 0.0015000000000000002
Node: 2, PageRank: 0.003982161009199527
Node: 97, PageRank: 0.0044699953032772905
Node: 40, PageRank: 0.009868077642541044
Node: 21, PageRank: 0.003333141463393135
Node: 59, PageRank: 0.004198750000000001
Node: 78, PageRank: 0.0015000000000000002
Node: 41, PageRank: 0.0038800389615251474
Node: 98, PageRank: 0.00469231217068975
Node: 79, PageRank: 0.0056705271640864335
Node: 3, PageRank: 0.004819490969580691
Node: 22, PageRank: 0.0015000000000000002
Node: 60, PageRank: 0.0015000000000000002
Node: 42, PageRank: 0.002066634053410217
Node: 4, P

# 3. Una estrategia general

1. Preparación de los nodos:

Inicializar las propiedades de los nodos, como el valor de PageRank o la distancia más corta en el caso de SSSP.
Asignar un identificador único a cada nodo del grafo.

2. Regla para pasar mensajes entre nodos:

En cada iteración del algoritmo, cada nodo envía mensajes a sus vecinos para compartir información.
Los mensajes pueden contener información relevante para el cálculo, como el valor de PageRank actualizado o la distancia más corta encontrada.
Los nodos reciben los mensajes enviados por sus vecinos y los utilizan para actualizar su información local.

3. Definición de funciones para hacer merge de varios mensajes:

Los nodos pueden recibir múltiples mensajes de diferentes vecinos en una iteración.
Se debe definir una función de merge que combine los mensajes recibidos en uno solo, considerando la información relevante para el cálculo.
Por ejemplo, en el caso de PageRank, los mensajes contienen contribuciones de los vecinos al PageRank del nodo receptor, y la función de merge sumaría estas contribuciones para obtener el nuevo valor de PageRank.

4. Actualización de las propiedades de los nodos:

Después de cada iteración, los nodos actualizan sus propiedades locales con la información recibida de los mensajes.
Por ejemplo, en el caso de PageRank, los nodos actualizan su valor de PageRank con la suma de las contribuciones recibidas de los vecinos.

5. Condiciones de término del algoritmo:

Se debe establecer un criterio de convergencia para determinar cuándo el algoritmo ha terminado.
Por ejemplo, en el caso de PageRank, se puede definir un umbral de cambio máximo en los valores de PageRank de los nodos.
Si la diferencia entre los valores de PageRank de dos iteraciones consecutivas es menor que el umbral, se considera que el algoritmo ha convergido y se detiene.






Cambiamos de orden el 4 y el 5